이 튜토리얼에서는 summarize 함수의 각 부분을 나눠서 직접 실행합니다. 데이터는 라라랜드 영화 리뷰를 이용합니다.

In [1]:
import krwordrank
print(krwordrank.__version__)

# La La Land
fname = '../data/134963_norm.txt'

def get_texts_scores(fname):
    with open(fname, encoding='utf-8') as f:
        docs = [doc.lower().replace('\n','').split('\t') for doc in f]
        docs = [doc for doc in docs if len(doc) == 2]
        
        if not docs:
            return [], []
        
        texts, scores = zip(*docs)
        return list(texts), list(scores)

texts, scores = get_texts_scores(fname)

1.0.1


KR-WordRank 를 이용하여 키워드를 학습합니다.

In [2]:
from krwordrank.word import KRWordRank

wordrank_extractor = KRWordRank(
    min_count = 5, # 단어의 최소 출현 빈도수 (그래프 생성 시)
    max_length = 10, # 단어의 최대 길이
    verbose = True
    )

beta = 0.85    # PageRank의 decaying factor beta
max_iter = 10

keywords, rank, graph = wordrank_extractor.extract(texts, beta, max_iter, num_keywords=100)

scan vocabs ... 
num vocabs = 13879
done = 9 Early stopped.


make_vocab_score 함수는 keywords 와 stopwords 를 이용하여 MaxScoreTokenizer 가 이용하는 단어 점수를 만드는 과정입니다.

MaxScoreTokenizer 는 [soynlp](https://github.com/lovit/soynlp/) 의 토크나이저 입니다.

In [3]:
from krwordrank.sentence import make_vocab_score
from krwordrank.sentence import MaxScoreTokenizer


stopwords = {'영화', '관람객', '너무', '정말', '진짜'}
vocab_score = make_vocab_score(keywords, stopwords, scaling=lambda x:1)
tokenizer = MaxScoreTokenizer(vocab_score)

Key sentence 를 추출하는데 필요한 정보는 keyword 뿐이기 때문에 토크나이징이 아주 정교하게 작동하지는 않습니다. 문장에서 키워드를 단어로 추출하는 역할만 합니다.

In [4]:
tokenizer.tokenize('뮤지컬영화라그런지 음악이좋다 그리고 엔딩이정말 먹먹하다')

['뮤지컬', '영화라', '그런', '지', '음악', '이좋다', '그리고', '엔딩', '이정말', '먹먹하다']

penalty 함수를 설정하고, 이들을 keysentence 함수에 입력합니다. 여기에서의 topk 는 핵심 문장의 개수입니다.

In [5]:
from krwordrank.sentence import keysentence

penalty = lambda x: 0 if 25 <= len(x) <= 80 else 1

sents = keysentence(
    vocab_score, texts, tokenizer.tokenize,
    penalty=penalty,
    diversity=0.3,
    topk=10
)

for sent in sents:
    print(sent)

사랑 꿈 현실 모든걸 다시한번 생각하게 하는 영화였어요 영상미도 너무 예쁘고 주인공도 예쁘고 내용도 아름답네요ㅠㅠ 인생 영화
생각보다 굉장히 재미있는 뻔한 결말도 아니고 아름다운 음악과 현실적인 스토리구성 모두에게 와닿을법한 울림들이 차 좋았어요 추천
남자친구랑 봤는데 진짜 다시 보고싶음 ㅠㅠㅠ너무 좋았어요 재즈좋아하고 뮤지컬같은거 좋아하는사람들한텐 취저영화
노래도 좋고 영상미도 좋고 그리고 배우들 연기까지 정말 좋았어요 개인적으로 뮤지컬 형식 영화를 안좋아하는 편인데 재밌게 봤습니다
영화같은 사랑 현실적인 결말 마지막 장면처럼 모든 것이 원하는 대로 슬픈 일 하나없이 흘러갈 수는 없는 것이 인생
영상 음악 연출 연기 모든게 만점입니다 너무 현실적이라 슬프고 눈물이 나네요 라이언 고슬링이 이렇게 연기를 잘 하는 배우인지 처음 알았네요
음악도 좋고 미아와 세바스티안의 아름다운 사랑과 예술에 대한 열정이 감동적이었습니다 재즈음악을 사랑하고 뮤지컬을 좋아하는 사람들에게 강추합니다
음악과 영상미 모두좋았습니다 특히 마지막 10분은 가히압권이였습니다 이런좋은영화 많이보았으면좋겠네요 ㅎㅎ
처음 써보는 영화에대한 평점 음악부터 연기 배경 그리고 색감 모든게 마음에 들었으며 나의 인생영화가된 영화
보는내내 마음이 따뜻해지는 영화네요 노래도 좋고 좋아하는 사람과 함께 봐서 더 좋았던것 같아요 연인과 함께 보는것 추천합니다


위의 예시에서 diversity 를 키우면 핵심 문장 점수가 높은 문장과 cosine distance 가 diversity 보다 작고 점수가 낮은 문장은 선택되지 않습니다.

In [6]:
sents = keysentence(
    vocab_score, texts, tokenizer.tokenize,
    penalty=penalty,
    diversity=0.7,
    topk=10
)

for sent in sents:
    print(sent)

사랑 꿈 현실 모든걸 다시한번 생각하게 하는 영화였어요 영상미도 너무 예쁘고 주인공도 예쁘고 내용도 아름답네요ㅠㅠ 인생 영화
생각보다 굉장히 재미있는 뻔한 결말도 아니고 아름다운 음악과 현실적인 스토리구성 모두에게 와닿을법한 울림들이 차 좋았어요 추천
남자친구랑 봤는데 진짜 다시 보고싶음 ㅠㅠㅠ너무 좋았어요 재즈좋아하고 뮤지컬같은거 좋아하는사람들한텐 취저영화
음악과 영상미 모두좋았습니다 특히 마지막 10분은 가히압권이였습니다 이런좋은영화 많이보았으면좋겠네요 ㅎㅎ
처음 써보는 영화에대한 평점 음악부터 연기 배경 그리고 색감 모든게 마음에 들었으며 나의 인생영화가된 영화
보는 내내 두근두근 어느 순간도 눈을 뗄수 없는 환상적인 영상과 음악 현실성 높은 스토리에 배우들의 멋진 연기까지 행복한 영화였어요
마지막 장면에서 라이언고슬링의 피아노 연주와 엠마스톤의 눈빛연기 그리고 두 사람이 함께 했다면 어땠을까 하는 상상씬에서의 연출이 인상적이었다
인생영화 노래 연기 내용 연출이 다 엄청났다 ㅠㅠ 꿈을 위해 노력하고있는 사람에게 도움이 많이 될것같다
정말 여자들이 좋아할 영화에요 영상이나 ost가 정말 예술이에요 배우들의 노래도 하나하나 다 좋았어요 마지막에 스토리가 좀 아쉽긴 하지만
감동과 여운이 남는 영화네요 배우들 연기는 물론 음악과 배경까지 너무 좋아요 최근에 본 영화 중에 가장 좋았습니다 추천이요


summarize 함수는 위 과정을 한 번에 실행합니다. topk 가 키워드와 문장에 모두 적용되어야 하기 때문에 num_keywords 와 num_sents 로 설정합니다.

In [7]:
from krwordrank.sentence import summarize_with_sentences


penalty = lambda x:0 if (25 <= len(x) <= 80) else 1
stopwords = {'영화', '관람객', '너무', '정말', '진짜'}

keywords, sents = summarize_with_sentences(
    texts,
    penalty=penalty,
    stopwords = stopwords,
    diversity=0.7,
    num_keywords=100,
    num_keysents=10,
    scaling=lambda x:1,
    verbose=False,
)

for sent in sents:
    print(sent)

사랑 꿈 현실 모든걸 다시한번 생각하게 하는 영화였어요 영상미도 너무 예쁘고 주인공도 예쁘고 내용도 아름답네요ㅠㅠ 인생 영화
생각보다 굉장히 재미있는 뻔한 결말도 아니고 아름다운 음악과 현실적인 스토리구성 모두에게 와닿을법한 울림들이 차 좋았어요 추천
남자친구랑 봤는데 진짜 다시 보고싶음 ㅠㅠㅠ너무 좋았어요 재즈좋아하고 뮤지컬같은거 좋아하는사람들한텐 취저영화
인생영화 노래 연기 내용 연출이 다 엄청났다 ㅠㅠ 꿈을 위해 노력하고있는 사람에게 도움이 많이 될것같다
음악과 영상미 모두좋았습니다 특히 마지막 10분은 가히압권이였습니다 이런좋은영화 많이보았으면좋겠네요 ㅎㅎ
처음 써보는 영화에대한 평점 음악부터 연기 배경 그리고 색감 모든게 마음에 들었으며 나의 인생영화가된 영화
마지막 회상신에서 눈물이 왈칵 쏟아질뻔했다 올해중 최고의 영화를 본거 같다음악이며 배우들이며 영상이며 다시 또 보고싶은 그런 영화이다
보는 내내 두근두근 어느 순간도 눈을 뗄수 없는 환상적인 영상과 음악 현실성 높은 스토리에 배우들의 멋진 연기까지 행복한 영화였어요
마지막 장면에서 라이언고슬링의 피아노 연주와 엠마스톤의 눈빛연기 그리고 두 사람이 함께 했다면 어땠을까 하는 상상씬에서의 연출이 인상적이었다
정말 여자들이 좋아할 영화에요 영상이나 ost가 정말 예술이에요 배우들의 노래도 하나하나 다 좋았어요 마지막에 스토리가 좀 아쉽긴 하지만
